In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import os

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
COLUMN_OF_INTEREST =['QUOTE_UNIXTIME','QUOTE_DATE','EXPIRE_DATE','EXPIRE_UNIX',
                            'DTE','C_BID','C_ASK', 'P_BID','P_ASK',
                             'UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE']

In [3]:
# def read_data(path):
#     df = pd.read_csv(path)

#     df.columns = df.columns.str.replace(' ', '')
#     df.columns = df.columns.str.replace('[', '', regex=True)
#     df.columns = df.columns.str.replace(']', '', regex=True)

#     df_of_interest = df[COLUMN_OF_INTEREST]
#     df_of_interest = df_of_interest[(df_of_interest['DTE'] == 30) | (df_of_interest['DTE'] == 0)]

#     return df_of_interest

In [4]:
# # create empty dataframe with column names
# df_of_interest = pd.DataFrame(columns=COLUMN_OF_INTEREST)

# #loop through all files in directory
# for filename in tqdm(os.listdir('data/2020 to 2022 txt files')):
#     if not filename.endswith(".txt"):
#         continue
#     path = os.path.join('./data/2020 to 2022 txt files', filename)
#     df_of_interest = pd.concat([df_of_interest, read_data(path)])


In [5]:
# write to csv
# df_of_interest.to_csv('data/2020-2022_30days.csv', index=False)

In [6]:
df_options = pd.read_csv('data/2020-2022_30days.csv')

# convert QUOTE_DATE and EXPIRE_DATE to datetime
df_options['QUOTE_DATE'] = pd.to_datetime(df_options['QUOTE_DATE'])
df_options['EXPIRE_DATE'] = pd.to_datetime(df_options['EXPIRE_DATE'])

# drop QUOTE_UNIXTIME and EXPIRE_UNIX
df_options = df_options.drop(['QUOTE_UNIXTIME', 'EXPIRE_UNIX'], axis=1)

# set "" to NaN
df_options = df_options.replace(r'^\s*$', np.nan, regex=True)

# drop rows with NaN
df_options = df_options.dropna()

# convert C_BID, C_ASK, P_BID, P_ASK to float
df_options['C_BID'] = df_options['C_BID'].astype(float)
df_options['C_ASK'] = df_options['C_ASK'].astype(float)
df_options['P_BID'] = df_options['P_BID'].astype(float)
df_options['P_ASK'] = df_options['P_ASK'].astype(float)

In [7]:
df_options.dtypes

QUOTE_DATE         datetime64[ns]
EXPIRE_DATE        datetime64[ns]
DTE                       float64
C_BID                     float64
C_ASK                     float64
P_BID                     float64
P_ASK                     float64
UNDERLYING_LAST           float64
STRIKE                    float64
STRIKE_DISTANCE           float64
dtype: object

In [8]:
df_options.head()

,QUOTE_DATE,EXPIRE_DATE,DTE,C_BID,C_ASK,P_BID,P_ASK,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE
0,2020-01-03,2020-01-03,0.0,1826.61,1849.20,0.0,0.05,3234.35,1400.0,1834.3
1,2020-01-03,2020-01-03,0.0,1726.60,1749.30,0.0,0.05,3234.35,1500.0,1734.3
2,2020-01-03,2020-01-03,0.0,1626.60,1649.19,0.0,0.04,3234.35,1600.0,1634.3
3,2020-01-03,2020-01-03,0.0,1526.60,1549.31,0.0,0.05,3234.35,1700.0,1534.3
4,2020-01-03,2020-01-03,0.0,1426.60,1449.30,0.0,0.04,3234.35,1800.0,1434.3


In [109]:
# Calculate the rolling volatility of the underlying price
import yfinance as yf
from pypfopt.risk_models import CovarianceShrinkage

TICKER = ['SPY']

start_date = '2017-01-01' # you need to have at least 2 years of data before the start date as we are calculating the rolling volatility of 2 years
end_date = '2023-01-01' # end date is exclusive, the data will be downloaded until 2022-12-31

df = yf.download(TICKER, start=start_date, end=end_date)

# calculate the rolling volatility of 2 years (252 trading days per year)
# rolling_period = 252*2
rolling_period = int(252)
# dt = 1/252

# VOLATILITY METHOD 1: calculate the volatility using log return and take the standard deviation as is
# df['log_return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
# df['volatility'] = df['log_return'].rolling(rolling_period).std() * np.sqrt(252)

# VOLATILITY METHOD 2: calculate the volatility using log return and CovarianceShrinkage
df_spy = df['Adj Close']
df_spy.index.name = None
df['volatility'] = np.nan

#Apply rolling period of 2 years to CovarianceShrinkage
for i in range(rolling_period, len(df_spy)):
    cov_matrix = CovarianceShrinkage(df_spy.iloc[i-rolling_period:i], log_returns=True).ledoit_wolf() 

    #calculate the volatility. cov_matrix is annualized, so no need to multiply by sqrt(252)
    df['volatility'].iloc[i] = np.sqrt(cov_matrix.iloc[0,0])
    
# we only need the data from 2020 to 2022
df = df.loc['2020-01-01':'2022-12-31']

[*********************100%***********************]  1 of 1 completed


In [110]:
# risk free rate for 2020 to 2022 is around 2.5% per annum
risk_free_rate = 0.025

# dividend yield is rate is roughly 1.5% per annum for SPY
dividend_yield = 0.015

In [111]:
def binomial_model(S0, K, T, r, sigma, N, 
                   option='call', div=0, 
                   european=True, method='crr'):
    ''' Binomial model for option pricing.
    Parameters
    ==========
    S0 : float
        initial stock/index level
    K : float
        strike price
    T : float
        maturity (in year fractions)
    r : float
        constant risk-free short rate
    sigma : float
        volatility factor in diffusion term 
    N : int
        number of time intervals
    option : string
        type of the option to be valued ('call' or 'put')
    div : float
        annualized continuous dividend yield
    european : boolean
        True for European option valuation
        False for American option valuation
    method : string
        'crr' for Cox, Ross and Rubinstein
        'jr' for Jarrow and Rudd/equal probabilities 
    Returns
    =======
    value : float
        present value of the European option
    '''
    dt = T / N
    
    # calculate the risk-neutral probability
    if method == 'crr':
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp((r - div) * dt) - d) / (u - d)
    elif method == 'jr':
        u = np.exp((r-div-0.5*sigma**2)*dt+sigma*np.sqrt(dt))
        d = np.exp((r-div-0.5*sigma**2)*dt-sigma*np.sqrt(dt))
        p = 0.5
    else:
        raise ValueError('method not recognized')

    q = 1 - p

    # initialize terminal stock prices at maturity
    stock_prices = np.zeros((N + 1, N + 1))
    stock_prices[0, 0] = S0
    for i in range(1, N + 1):
        stock_prices[0, i] = stock_prices[0, i - 1] * u
        for j in range(1, i + 1):
            stock_prices[j, i] = stock_prices[j - 1, i - 1] * d

    # calculate option values at maturity
    option_values = np.zeros((N + 1, N + 1))
    if option == 'call':
        option_values[:, N] = np.maximum(stock_prices[:, N] - K, 0)
    else:
        option_values[:, N] = np.maximum(K - stock_prices[:, N], 0)

    # backward induction
    if european:
        for i in range(N - 1, -1, -1):
            for j in range(i + 1):
                option_values[j, i] = np.exp(-r * dt) * (p * option_values[j, i + 1] + q * option_values[j + 1, i + 1])
    else:
        if option == 'call':
            for i in range(N - 1, -1, -1):
                for j in range(i + 1):
                    option_values[j, i] = np.exp(-r * dt) * (p * option_values[j, i + 1] + q * option_values[j + 1, i + 1])
                    option_values[j, i] = np.maximum(option_values[j, i], stock_prices[j, i] - K)
        else:
            for i in range(N - 1, -1, -1):
                for j in range(i + 1):
                    option_values[j, i] = np.exp(-r * dt) * (p * option_values[j, i + 1] + q * option_values[j + 1, i + 1])
                    option_values[j, i] = np.maximum(option_values[j, i], K - stock_prices[j, i])
    
    return option_values[0, 0]

In [112]:
S0 = 3246.23
K = 2000
T = 21/252
r = 0.025
div = 0.015
sigma = 0.1498878078948466
N = 21
print(binomial_model(S0, K, T, r, sigma, N, option='call', div=div, european=True, method='crr'))
print(binomial_model(S0, K, T, r, sigma, N, option='call', div=div, european=True, method='jr'))

1246.3370769621977
1246.3370318672155


In [113]:
# merge the dataframes of df_options and df where df_options QUOTE_DATE is same as df index
df_model_input = df_options.merge(df['volatility'], left_on='QUOTE_DATE', right_on=df.index)
df_model_input = df_model_input[df_model_input['DTE']==30]

In [114]:
df_model_input.head()

,QUOTE_DATE,EXPIRE_DATE,DTE,C_BID,C_ASK,P_BID,P_ASK,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,volatility
382,2020-01-06,2020-02-05,30.0,1241.90,1248.39,0.00,0.14,3246.23,2000.0,1246.2,0.119088
383,2020-01-06,2020-02-05,30.0,1142.89,1148.59,0.10,0.16,3246.23,2100.0,1146.2,0.119088
384,2020-01-06,2020-02-05,30.0,1092.10,1098.70,0.10,0.20,3246.23,2150.0,1096.2,0.119088
385,2020-01-06,2020-02-05,30.0,1043.10,1048.80,0.05,0.20,3246.23,2200.0,1046.2,0.119088
386,2020-01-06,2020-02-05,30.0,993.20,998.91,0.10,0.21,3246.23,2250.0,996.2,0.119088


In [115]:
# backtest the binomial model
# track the progress of the backtest
tqdm().pandas()
N_PLUS=0
df_model_input['binomial_model'] = df_model_input.progress_apply(
    lambda x: binomial_model(x['UNDERLYING_LAST'], x['STRIKE'], 
                             x['DTE']/365, risk_free_rate, x['volatility'], 
                             int(x['DTE'])+N_PLUS, option='call', div=dividend_yield, 
                             european=True, method='crr'), axis=1)

0it [00:00, ?it/s]

  0%|          | 0/38928 [00:00<?, ?it/s]

In [116]:
df_model_input['binomial_model_jr'] = df_model_input.progress_apply(
    lambda x: binomial_model(x['UNDERLYING_LAST'], x['STRIKE'], 
                             x['DTE']/365, risk_free_rate, x['volatility'], 
                             int(x['DTE'])+N_PLUS, option='call', div=dividend_yield, 
                             european=True, method='jr'), axis=1)


  0%|          | 0/38928 [00:00<?, ?it/s]

In [117]:
df_model_input.head()

,QUOTE_DATE,EXPIRE_DATE,DTE,C_BID,C_ASK,P_BID,P_ASK,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,volatility,binomial_model,binomial_model_jr
382,2020-01-06,2020-02-05,30.0,1241.90,1248.39,0.00,0.14,3246.23,2000.0,1246.2,0.119088,1246.335634,1246.335622
383,2020-01-06,2020-02-05,30.0,1142.89,1148.59,0.10,0.16,3246.23,2100.0,1146.2,0.119088,1146.540903,1146.540891
384,2020-01-06,2020-02-05,30.0,1092.10,1098.70,0.10,0.20,3246.23,2150.0,1096.2,0.119088,1096.643537,1096.643525
385,2020-01-06,2020-02-05,30.0,1043.10,1048.80,0.05,0.20,3246.23,2200.0,1046.2,0.119088,1046.746171,1046.746159
386,2020-01-06,2020-02-05,30.0,993.20,998.91,0.10,0.21,3246.23,2250.0,996.2,0.119088,996.848806,996.848793


In [118]:
df_eval = df_model_input[['QUOTE_DATE', 'binomial_model', 'binomial_model_jr', 'C_BID','C_ASK']]
df_eval['C_PRICE'] = (df_eval['C_BID'] + df_eval['C_ASK'])/2
df_eval = df_eval.set_index('QUOTE_DATE')
df_eval.head()

,binomial_model,binomial_model_jr,C_BID,C_ASK,C_PRICE
QUOTE_DATE,,,,,
2020-01-06,1246.335634,1246.335622,1241.90,1248.39,1245.145
2020-01-06,1146.540903,1146.540891,1142.89,1148.59,1145.740
2020-01-06,1096.643537,1096.643525,1092.10,1098.70,1095.400
2020-01-06,1046.746171,1046.746159,1043.10,1048.80,1045.950
2020-01-06,996.848806,996.848793,993.20,998.91,996.055


In [119]:
#calculate percentage of binomial model that is between bid and ask
df_eval['binomial_model_between_bid_ask'] = np.where((df_eval['binomial_model'] > df_eval['C_BID']) & (df_eval['binomial_model'] < df_eval['C_ASK']), 1, 0)
df_eval['binomial_model_jr_between_bid_ask'] = np.where((df_eval['binomial_model_jr'] > df_eval['C_BID']) & (df_eval['binomial_model_jr'] < df_eval['C_ASK']), 1, 0)

#calculate percentage of binomial model that is between bid and ask
print(df_eval['binomial_model_between_bid_ask'].mean())

#calculate percentage of binomial model that is between bid and ask
print(df_eval['binomial_model_jr_between_bid_ask'].mean())



0.2576037813399096
0.25778360049321825


In [120]:
#calculate absolute error of binomial model
df_eval['binomial_model_error'] = df_eval['binomial_model'] - df_eval['C_PRICE']
df_eval['binomial_model_jr_error'] = df_eval['binomial_model_jr'] - df_eval['C_PRICE']

#calculate root mean squared error of binomial model
print('Root Mean Squared Error of CRR model:',np.sqrt(np.mean(df_eval['binomial_model_error']**2)))
print('Root Mean Squared Error of JR model:',np.sqrt(np.mean(df_eval['binomial_model_jr_error']**2)))

#calculate mean absolute error of binomial model
print('Mean Absolute Error of CRR model:',np.mean(np.abs(df_eval['binomial_model_error'])))
print('Mean Absolute Error of JR model:',np.mean(np.abs(df_eval['binomial_model_jr_error'])))

#calculate mean absolute percentage error of binomial model
print('Percentage Mean Absolute Error of CRR model:', np.mean(np.abs(df_eval['binomial_model_error']/df_eval['C_PRICE']))*100,"%")
print('Percentage Mean Absolute Error of JR model:', np.mean(np.abs(df_eval['binomial_model_jr_error']/df_eval['C_PRICE']))*100,"%")


Root Mean Squared Error of CRR model: 20.894498703079908
Root Mean Squared Error of JR model: 20.89257775517791
Mean Absolute Error of CRR model: 12.17950212024097
Mean Absolute Error of JR model: 12.178586720863196
Percentage Mean Absolute Error of CRR model: 89.90164885572919 %
Percentage Mean Absolute Error of JR model: 89.7479419600846 %


In [121]:
#find row where binomial model is furthest to the actual price
df_eval[np.abs(df_eval['binomial_model_error']).max()==np.abs(df_eval['binomial_model_error'])]

,binomial_model,binomial_model_jr,C_BID,C_ASK,C_PRICE,binomial_model_between_bid_ask,binomial_model_jr_between_bid_ask,binomial_model_error,binomial_model_jr_error
QUOTE_DATE,,,,,,,,,
2020-03-16,82.506063,82.543239,225.29,242.2,233.745,0,0,-151.238937,-151.201761


In [122]:
# METHOD 1: using std as is
# Root Mean Squared Error of CRR model:  23.86935498966928
# Root Mean Squared Error of JR model:  25.103188203551444
# Mean Absolute Error of CRR model:  14.540254685604587
# Mean Absolute Error of JR model:  16.29314434180136
# Percentage of Error of CRR model:  3.081296435831501
# Percentage of Error of JR model:  3.4527598501135346


# METHOD 2: using COVARIANCE_SHRINKAGE
# Root Mean Squared Error of CRR model:  23.920759846458463
# Root Mean Squared Error of JR model:  25.15368471620861
# Mean Absolute Error of CRR model:  14.557521681224893
# Mean Absolute Error of JR model:  16.30927181380145
# Percentage of Error of CRR model:  3.0849555692657344
# Percentage of Error of JR model:  3.456177501527993